In [1]:
import pandas as pd

In [2]:
df_0 = pd.read_csv('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/Mario_report/data/2026-01-09T11-54_papers.csv')

In [3]:
df = df_0[['authors_full']]

In [4]:
df

,authors_full
0,"Mattia Stival, Lorenzo Schiavon, Gaia Bertarel..."
1,"Nicolas Chopin, Francesca R Crucinio, Sumeetpa..."
2,"Roberto Boselli, Simone D’Amico, Navid Nobani"
3,"Lorenzo Schiavon, Mattia Stival"
4,Lorenzo Schiavon
...,...
1122,"Michela Cinquini, Nicola Rocco, Giuseppe Catan..."
1123,"Peer Christiansen, Marco Mele, Anne Bodilsen, ..."
1124,"Yazan A Masannat, Nicola Rocco, Emanuele Garre..."
1125,"G. Catanuto, N. Rocco, A. Maglia, P. Barry, A...."


In [5]:
ssd = pd.read_excel('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/Mario_report/data/SSD_V3_area.xlsx')
cen = pd.read_csv('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/Mario_report/data/processed/censimento.csv')

In [6]:
ssd

,Cod. SSD,Area,Area_desc
0,MATH-01/A,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
1,MATH-01/B,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
2,MATH-02/A,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
3,MATH-02/B,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
4,MATH-03/A,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
...,...,...,...
322,GSPS-06/A,AREA 14,SCIENZE POLITICHE E SOCIALI
323,GSPS-07/A,AREA 14,SCIENZE POLITICHE E SOCIALI
324,GSPS-07/B,AREA 14,SCIENZE POLITICHE E SOCIALI
325,GSPS-08/A,AREA 14,SCIENZE POLITICHE E SOCIALI


In [7]:
cen

,Nome,Cognome,Codice fiscale,SSD,Email,age
0,Daniela,Trisciuoglio,TRSDNL73R55H501N,BIOS-10/A,daniela.trisciuoglio@cnr.it,53
1,Serena,Sanna,SNNSRN80S55H856X,BIOS-04/A,serena.sanna@cnr.it,46
2,Francesca,Cavalcanti,CVLFNC61T56D086E,BIOS-11/A,francesca.cavalcanti@irib.cnr.it,65
3,Paolo,Barsocchi,BRSPLA78S27G702C,INFO-01/A,paolo.barsocchi@isti.cnr.it,48
4,Antonino,Colanzi,CLNNNN64B29E435X,BIOS-10/A,antonino.colanzi@cnr.it,62
...,...,...,...,...,...,...
698,Francesco Paolo,Fanizzi,FNZFNC56S07C975L,CHEM-03/A,francesco.fanizzi@unisalento.it,70
699,Biagio,Iacolare,CLRBGI97M15F839A,STAT-03/A,biagio.iacolare@unina.it,29
700,Marta,Massei,MSSMRT93A69G843T,PSIC-01/B,marta.massei@unifi.it,33
701,Venturina,Stagnu,STGVTR77P46H501I,BIOS-14/A,venturina.stagni@cnr.it,49


In [8]:
cen = pd.merge(cen, ssd, left_on='SSD', right_on='Cod. SSD', how='left')

In [9]:
cen = cen.dropna(subset=['Area']).reset_index(drop=True)
cen['full_name'] = cen['Nome'] + ' ' + cen['Cognome']

In [10]:
cen = cen[['Area_desc', 'full_name']]

In [11]:
cen.to_csv('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/Mario_report/data/processed/chord_area.csv', index=False)

In [12]:
df.to_csv('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/Mario_report/data/processed/chord_authors.csv', index=False)

In [13]:
for i in df['authors_full'].tolist():
    names = i.lower().split(',')
    names = [i.strip() for i in names]

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

# --- Inputs ---
# cen: DataFrame with columns ["Area_desc", "full_name"]
# df:  DataFrame with column  ["authors_full"] where authors are comma-separated in a string

# --- 1) Build person -> field map ---
cen_clean = cen.copy()

cen_clean["full_name"] = (
    cen_clean["full_name"]
    .astype(str)
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

cen_clean["Area_desc"] = (
    cen_clean["Area_desc"]
    .astype(str)
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# If duplicates exist, keep the first occurrence (you can change this policy if needed)
name_to_field = (
    cen_clean.dropna(subset=["full_name", "Area_desc"])
             .drop_duplicates(subset=["full_name"], keep="first")
             .set_index("full_name")["Area_desc"]
             .to_dict()
)

# --- 2) Parse authors list per paper ---
def parse_authors(s: str) -> list[str]:
    if pd.isna(s):
        return []
    # split by comma; normalise whitespace; drop empties
    out = [a.strip() for a in str(s).split(",")]
    out = [a for a in out if a]
    return out

df_auth = df.copy()
df_auth["authors_list"] = df_auth["authors_full"].apply(parse_authors)

# Map each author to a field (unknowns become NaN)
df_auth["fields_list"] = df_auth["authors_list"].apply(
    lambda authors: [name_to_field.get(a, np.nan) for a in authors]
)

# Optionally drop unknown-field authors
df_auth["fields_list"] = df_auth["fields_list"].apply(
    lambda fs: [f for f in fs if pd.notna(f)]
)

# --- 3) Count collaborations and build matrix ---
def collaboration_matrix(df_auth: pd.DataFrame, mode: str = "pairwise") -> pd.DataFrame:
    """
    mode:
      - "pairwise": counts all co-author pairs; large teams contribute more
      - "paper_level": counts each field-field pair at most once per paper
    """
    counts = {}

    for fields in df_auth["fields_list"]:
        if len(fields) < 2:
            continue

        if mode == "pairwise":
            # create all author-author pairs, then aggregate by their fields
            for f1, f2 in combinations(fields, 2):
                a, b = sorted((f1, f2))
                counts[(a, b)] = counts.get((a, b), 0) + 1

        elif mode == "paper_level":
            # only unique fields per paper; count each field-field pair once per paper
            uniq = sorted(set(fields))
            for f1, f2 in combinations(uniq, 2):
                counts[(f1, f2)] = counts.get((f1, f2), 0) + 1

        else:
            raise ValueError("mode must be 'pairwise' or 'paper_level'")

    # Collect all fields that appear in collaborations
    fields_all = sorted(set([x for pair in counts.keys() for x in pair]))

    mat = pd.DataFrame(0, index=fields_all, columns=fields_all, dtype=int)

    for (a, b), v in counts.items():
        mat.loc[a, b] += v
        mat.loc[b, a] += v  # symmetric

    return mat

# Build the matrix (choose mode)
mat_pairwise = collaboration_matrix(df_auth, mode="pairwise")
mat_paperlvl = collaboration_matrix(df_auth, mode="paper_level")

# If you want a D3-ready "matrix" (list of lists) + labels:
labels = mat_pairwise.index.tolist()
matrix = mat_pairwise.values.tolist()

In [ ]:
matrix

[[22, 2, 0, 0, 0, 1, 0, 0, 15, 2, 0, 3],
 [2, 126, 17, 3, 3, 33, 9, 0, 23, 82, 8, 18],
 [0, 17, 176, 4, 1, 21, 11, 0, 6, 136, 4, 7],
 [0, 3, 4, 48, 1, 1, 7, 0, 6, 23, 0, 3],
 [0, 3, 1, 1, 0, 1, 2, 0, 6, 14, 0, 3],
 [1, 33, 21, 1, 1, 210, 2, 0, 6, 112, 15, 18],
 [0, 9, 11, 7, 2, 2, 4, 0, 12, 37, 0, 6],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 2],
 [15, 23, 6, 6, 6, 6, 12, 0, 170, 86, 0, 18],
 [2, 82, 136, 23, 14, 112, 37, 0, 86, 1340, 12, 75],
 [0, 8, 4, 0, 0, 15, 0, 7, 0, 12, 50, 18],
 [3, 18, 7, 3, 3, 18, 6, 2, 18, 75, 18, 92]]